This notebook is designed to be run either locally or in Google Colab. For usage in Google Colab, the following files/folders should be uploaded to the default directory:
- utils.py
- /data/
    
Resources:
- https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_clm.py
- https://github.com/huggingface/transformers/blob/master/examples/text-generation/run_generation.py
- https://huggingface.co/transformers/main_classes/trainer.html
- https://huggingface.co/transformers/training.html

# Install Prerequisites

In [ ]:
!mkdir data/bbc/tech
!mkdir src/

In [4]:
import os
import sys
if 'google.colab' in str(get_ipython()):
    !pip install datasets
    !pip install transformers
else:
    sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

from importlib import reload
import src.utils
reload(src.utils)

from src.utils import DataReader

# Load Data

In [5]:
from src.utils import DataReader
# from src.utils import pass_sliding_window

In [16]:
text = DataReader.read_bbc_tech()
text = ' '.join(sentence for sentence in text) # join into large string
text = text.split(' ') # split into words

/Users/jaipancholi/Code/autocomplete_me/src


# Load Model

In [11]:
import tensorflow as tf
from src.utils import TransformerLoader

In [12]:
model = 'gpt2'
tokenizer, model = TransformerLoader.from_huggingface(model, framework='tf')

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [13]:
type(model)

transformers.modeling_tf_gpt2.TFGPT2Model

# Prepare Data

In [43]:
text_as_single_string = ' '.join(text)
tokenized_text = tokenizer.encode(text_as_single_string, return_tensors='tf')
tokenized_text = tokenized_text[0]

In [45]:
# default tokeniser stems
for i in range(10):
    value = tokenized_text[i].numpy()
    print(text[i], value, tokenizer.decode([value]))

Mobiles 16540  Mob
rack 2915 iles
up 19127  rack
20 510  up
years 1160  20
of 812  years
use 286  of
 779  use
Mobile 220  
phones 12173  Mobile


In [47]:
# split into chunks
seq_length = 10

features = []
labels = []

# here labels == features
for i in range(0, len(tokenized_text) - seq_length, seq_length):
    features.append(tokenized_text[i:i+seq_length])
    labels.append(tokenized_text[i:i+seq_length])


# here labels = featurs + 1 (shifted by 1, next token prediction)
examples = []
for i in range(0, len(tokenized_text) - seq_length + 1, seq_length):
    examples.append(tokenized_text[i:i + seq_length])

for ex in examples:
    features.append(ex[:-1])
    labels.append(ex[1:])

    features.append(tokenized_text[i:i+seq_length])
    labels.append(tokenized_text[i+1:i+seq_length+1])

print(len(features[0]))
print(len(labels[0]))

10
10


In [48]:
BATCH_SIZE = 16
BUFFER_SIZE = 1000

dataset = tf.data.Dataset.from_tensor_slices((features, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [50]:
# train
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [51]:
num_epoch = 10
history = model.fit(dataset, epochs=num_epoch)
# Error Here

Epoch 1/10


ValueError: in user code:

    /Users/jaipancholi/Code/autocomplete_me/venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/jaipancholi/Code/autocomplete_me/venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/jaipancholi/Code/autocomplete_me/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/jaipancholi/Code/autocomplete_me/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/jaipancholi/Code/autocomplete_me/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_r

In [ ]:
model.predict(dataset).shape

In [179]:
[loss, *[None] * model.config.n_layer]

 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]